<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Vectorized Backtesting of Trading Strategies**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical end-of-day and intraday price data,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks`,
* how to derive typical trading statistics and formulate trading strategies and
* how to backtest such trading strategies in vectorized fashion.

## Simple Moving Averages

Trading based on **simple moving averages (SMAs)** is a decades old strategy that has its origins in the technical stock analysis world. Brock et al. (1992), for example, empirically investigate such strategies systematically. They write:

> "The term 'technical analysis' is a general heading for a myriad of trading techniques. ... In this paper, we explore two of the simplest and most popular technical rules: moving average-oscillator and trading-range break (resistance and support levels). In the first method, buy and sell signals are generated by two moving averages, a long period, and a short period. ... Our study reveals that technical analysis helps to predict stock changes."

See Brock, William, Josef Lakonishok, Blake LeBaron (1992): "Simple Technical Trading Rules and the Stochastic Properties of Stock Returns." _Journal of Finance_, Vol. 47, No. 5, 1731-1764.

In what follows, a **trading strategy based on two SMAs, a shorter and a longer one, is analyzed** &mdash; first based on end-of-day data then based on intraday data.

## Importing Required Packages

In [ ]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [ ]:
import sys
print(sys.version)

In [ ]:
ek.__version__

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
cf.__version__

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally.

In [ ]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')

In [ ]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## Retrieving End-of-Day Data

First, **end-of-day (EOD) data** for a stock is retrieved.

In [ ]:
ric = 'AAPL.O'

In [ ]:
data = ek.get_timeseries(ric,  # the RIC
                         fields='CLOSE',  # the required fields
                         start_date='2010-01-01',  # start date
                         end_date='2018-04-30')  # end date

In [ ]:
data.head()  # first five rows

In [ ]:
data.tail()  # final five rows

In [ ]:
data.info()  # DataFrame meta information

In [ ]:
data.normalize().iplot(kind='lines')

## Deriving the SMA Values

`pandas` provides convenience methods to calculate **rolling statistics**, such as SMAs, in a standardized way.

In [ ]:
SMA1 = 42  # shorter SMA window
SMA2 = 252  # longer SMA window

In [ ]:
data['SMA1'] = data['CLOSE'].rolling(SMA1).mean()
data['SMA2'] = data['CLOSE'].rolling(SMA2).mean()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.iplot()

It makes sense to **delete all incomplete data rows** before proceeding.

In [ ]:
data.dropna(inplace=True)

## Strategy Formulation

The basic idea behind SMA-based strategies is to be positioned long, when the shorter SMA is above the longer SMA and to be positioned short in the other case.

Representing a **long position** by `+1` and a **short position** by `-1`, the positions are then derived as follows:

In [ ]:
data['POSITIONS'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

Graphically, one gets the following **positionings over time**.

In [ ]:
data.iplot(secondary_y='POSITIONS')

## Vectorized Backtesting

The **log returns** build the basis for the vectorized backtesting.

In [ ]:
data['RETURNS'] = np.log(data['CLOSE'] / data['CLOSE'].shift(1))  # log returns in vectorized fashion

In [ ]:
data['RETURNS'].head()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data['RETURNS'].iplot(kind='histogram', subplots=True)

The next step is to derive the **returns of the strategy**, by mutliplying the positionings (shifted by one day to avoid a foresight bias) with the stock returns. I.e. a long position earns the stock return, a short position earns the negative value of the stock return.

In [ ]:
data['STRATEGY'] = data['POSITIONS'].shift(1) * data['RETURNS']

In [ ]:
data.dropna(inplace=True)

The final step is to add up the single resulting log returns over time and to apply the exponential functions to arrive at the **performance of the strategy compared to the benchmark of a passive investment in the stock**.

In [ ]:
np.exp(data[['RETURNS', 'STRATEGY']].sum())

The performance of the strategy can also be **compared over time** to the benchmark investment.

In [ ]:
data[['RETURNS', 'STRATEGY']].cumsum().apply(np.exp).iplot()

## Moving to Intraday Data

Let us repeat the analysis with intraday data and for a currency pair price.

In [ ]:
ric = 'EUR='

In [ ]:
data = ek.get_timeseries(ric,  # the RIC
                         fields='CLOSE',  # the required fields
                         start_date='2018-05-07 08:00:00',  # start time
                         end_date='2018-05-07 14:00:00',    # end time
                         interval='minute')

In [ ]:
data.head()  # first five rows

In [ ]:
data.info()  # DataFrame meta information

In [ ]:
data.normalize().iplot(kind='lines')

The **SMAs** derived and visualized.

In [ ]:
SMA1 = 10  # shorter SMA window
SMA2 = 30  # longer SMA window

In [ ]:
data['SMA1'] = data['CLOSE'].rolling(SMA1).mean()
data['SMA2'] = data['CLOSE'].rolling(SMA2).mean()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.iplot()

The **positionings** derived and visualized.

In [ ]:
data['POSITIONS'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

In [ ]:
data.iplot(secondary_y='POSITIONS')

The **log returns** calculated.

In [ ]:
data['RETURNS'] = np.log(data['CLOSE'] / data['CLOSE'].shift(1))  # log returns in vectorized fashion

In [ ]:
data.dropna(inplace=True)

Finally, the **strategy returns** and the **performance** compared to the benchmark passive investment.

In [ ]:
data['STRATEGY'] = data['POSITIONS'].shift(1) * data['RETURNS']

In [ ]:
data.dropna(inplace=True)

In [ ]:
np.exp(data[['RETURNS', 'STRATEGY']].sum())

In [ ]:
data[['RETURNS', 'STRATEGY']].cumsum().apply(np.exp).iplot()

## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical end-of-day as well as intraday price data** via the Eikon Data API,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient and
* `pandas` is a powerful data analysis tool to formulate and backtest **trading strategies** algorithmically and concisely based on **vectorized Python code**.

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">